# Knet CNN Example

In [3]:
using Pkg; haskey(Pkg.installed(),"Knet") || Pkg.add("Knet")
using Knet
True=true # so we can read the python params
include("common/params.py")
gpu()

0

In [4]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed()["Knet"])
println("GPU: ", read(`nvidia-smi --query-gpu=name --format=csv,noheader`,String))

OS: Linux
Julia: 1.0.0
Knet: 1.0.1+
GPU: Tesla K80



In [5]:
# define model
function initmodel(; atype=KnetArray, dtype=Float32, winit=xavier, binit=zeros)
    w(dims...)=atype(winit(dtype,dims...))
    b(dims...)=atype(binit(dtype,dims...))
    return Any[
        w(3,3,3,50), b(1,1,50,1),
        w(3,3,50,50), b(1,1,50,1),
        w(3,3,50,100), b(1,1,100,1),
        w(3,3,100,100), b(1,1,100,1),
        w(512,6400), b(512,1),
        w(10,512), b(10,1)
    ]
end;

In [6]:
# define loss and its gradient
function predict(w,x)
    convbias(x,w,b) = conv4(w,x;padding=1) .+ b
    fc(x,w,b) = w * mat(x) .+ b;
    x = relu.(convbias(x,w[1],w[2]))
    x = relu.(pool(convbias(x,w[3],w[4])))
    x = dropout(x,0.25)
    x = relu.(convbias(x,w[5],w[6]))
    x = relu.(pool(convbias(x,w[7],w[8])))
    x = dropout(x,0.25)
    x = relu.(fc(x,w[9],w[10]))
    x = dropout(x,0.5)
    return fc(x,w[11],w[12])
end

loss(w,x,y)=nll(predict(w,x),y) # nll: negative log likelihood
lossgradient = grad(loss);

In [7]:
# load data
include(Knet.dir("data","cifar.jl"))
@time (xtrn,ytrn,xtst,ytst,lbls)=cifar10()
for d in (xtrn,ytrn,xtst,ytst); println(summary(d)); end

┌ Info: Reading cifar-10-binary.tar.gz...
└ @ Main /kuacc/users/dyuret/.julia/dev/Knet/data/cifar.jl:41


  3.006594 seconds (3.14 M allocations: 1.869 GiB, 23.39% gc time)
32×32×3×50000 Array{Float32,4}
50000-element Array{UInt8,1}
32×32×3×10000 Array{Float32,4}
10000-element Array{UInt8,1}


In [8]:
# prepare for training
model = optim = nothing; Knet.gc() # Clear memory from last run
model = initmodel()
optim = optimizers(model, Momentum; lr=LR, gamma=MOMENTUM);

In [9]:
# cold start
@time for (x,y) in minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true,xtype=KnetArray)
    grads = lossgradient(model, x, y)
    update!(model, grads, optim)
end

 27.183593 seconds (19.80 M allocations: 1.549 GiB, 7.32% gc time)


In [10]:
# prepare for training
model = optim = nothing; Knet.gc() # Clear memory from last run
model = initmodel()
optim = optimizers(model, Momentum; lr=LR, gamma=MOMENTUM);

In [11]:
# 159s
@info("Training...")
@time for epoch in 1:EPOCHS
    @time for (x,y) in minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true,xtype=KnetArray)
        grads = lossgradient(model, x, y)
        update!(model, grads, optim)
    end
end

┌ Info: Training...
└ @ Main In[11]:2


 16.093201 seconds (1.87 M allocations: 664.531 MiB, 0.32% gc time)
 15.710396 seconds (1.87 M allocations: 664.347 MiB, 0.27% gc time)
 15.713468 seconds (1.87 M allocations: 664.345 MiB, 0.31% gc time)
 15.736086 seconds (1.87 M allocations: 664.342 MiB, 0.31% gc time)
 15.753533 seconds (1.87 M allocations: 664.394 MiB, 0.27% gc time)
 15.780440 seconds (1.87 M allocations: 664.347 MiB, 0.29% gc time)
 15.771320 seconds (1.87 M allocations: 664.345 MiB, 0.39% gc time)
 15.784548 seconds (1.87 M allocations: 664.461 MiB, 0.29% gc time)
 15.787313 seconds (1.87 M allocations: 664.403 MiB, 0.29% gc time)
 15.793624 seconds (1.87 M allocations: 664.403 MiB, 0.29% gc time)
157.932421 seconds (18.73 M allocations: 6.488 GiB, 0.30% gc time)


In [12]:
# test accuracy 77.54
testdata = minibatch(xtst,ytst,BATCHSIZE;xtype=KnetArray)
@time accuracy(model,testdata,predict)

  1.819726 seconds (1.69 M allocations: 200.570 MiB, 2.46% gc time)


0.7326722756410257